### API key

In [1]:
import os

## State

In [3]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

## ToolNode

In [7]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import ToolNode, tools_condition

tool = TavilySearchResults(max_results=2)
tools = [tool]

tool_node = ToolNode(tools)

### What is the Tool Node

In [9]:
import json
from langchain_core.messages import ToolMessage

class BasicToolNode:
    """A node that runs the tools requested in the last AIMessage."""

    def __init__(self, tools: list) -> None:
        self.tools_by_name = {tool.name: tool for tool in tools}

    def __call__(self, inputs: dict):
        if messages := inputs.get("messages", []):
            message = messages[-1]
        else:
            raise ValueError("No message found in input")
        outputs = []
        for tool_call in message.tool_calls:
            tool_result = self.tools_by_name[tool_call["name"]].invoke(tool_call["args"])
            outputs.append(
                ToolMessage(
                    content=json.dumps(tool_result),
                    name=tool_call["name"],
                    tool_call_id=tool_call["id"]
                )
            )
        return {"messages": outputs}

## LLM chatbot

In [10]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini')
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    result = llm_with_tools.invoke(state["messages"])
    return {"messages": [result]}

### Graph

In [11]:
from langgraph.graph import StateGraph

graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)

graph_builder.add_edge("tools", "chatbot")
graph_builder.add_conditional_edges("chatbot", tools_condition)

graph_builder.set_entry_point("chatbot")

graph = graph_builder.compile()

### Question that require internet search

In [12]:
graph.invoke({"messages": {"role": "user", "content": "지금 한국 대통령은 누구야?"}})

{'messages': [HumanMessage(content='지금 한국 대통령은 누구야?', additional_kwargs={}, response_metadata={}, id='115e2f73-5428-4b43-b81b-f17de6854787'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_WhKvsXnRhfsAiCcFxGBR2qI7', 'function': {'arguments': '{"query":"current president of South Korea 2023"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 87, 'total_tokens': 112, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-754138f7-796b-4125-8e01-5cb62c0ff64e-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current president of South Korea 2023'}, 'id':

### Questions an LLM can answer

In [13]:
graph.invoke({"messages": {"role": "user", "content": "마이크로소프트가 어떤 회사야?"}})

{'messages': [HumanMessage(content='마이크로소프트가 어떤 회사야?', additional_kwargs={}, response_metadata={}, id='2716ba70-d0f8-4990-a156-ca128b5f6d00'),
  AIMessage(content='마이크로소프트(Microsoft)는 미국의 다국적 기술 회사로, 소프트웨어, 전자기기, 컴퓨터 서비스 및 관련 제품을 개발하고 판매하는 기업입니다. 1975년에 빌 게이츠와 폴 앨런에 의해 설립되었으며, 본사는 워싱턴 주 레드몬드에 위치하고 있습니다. \n\n마이크로소프트는 주로 다음과 같은 분야에서 활동하고 있습니다:\n\n1. **소프트웨어**: 가장 유명한 제품은 운영 체제인 윈도우(Windows)와 오피스(Office) 제품군입니다. 이 외에도 클라우드 서비스인 애저(Azure)와 같은 다양한 소프트웨어 솔루션을 제공합니다.\n\n2. **하드웨어**: 서피스(Surface) 라인업의 노트북, 태블릿, 데스크톱 컴퓨터 및 게임 콘솔인 엑스박스(Xbox)도 출시하고 있습니다.\n\n3. **클라우드 컴퓨팅**: 기업 고객을 위한 클라우드 서비스와 솔루션을 제공하며, 이는 현재 마이크로소프트의 중요한 성장 분야 중 하나입니다.\n\n4. **게임**: 마이크로소프트는 게임 산업에서도 큰 역할을 하고 있으며, 엑스박스 게임 스튜디오 및 다양한 게임 타이틀을 개발하고 퍼블리싱합니다.\n\n마이크로소프트는 전 세계적으로 큰 영향력을 미치며, 기술 혁신과 디지털 전환을 주도하는 기업 중 하나로 알려져 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 330, 'prompt_tokens': 89, 'total_tokens': 419, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'a